In [184]:
#import json
import pickle
from pathlib import Path
from copy import deepcopy
import pandas as pd
#from collections import defaultdict
from datetime import datetime
from typing import Optional

from gnews import GNews
from newspaper import Article

In [144]:
COUNTRY = "PT"
LANGUAGE = "pt"

In [183]:
class NewsFinder():
    # names returned by GNews
    url_name = "url"
    title_name = "title"
    description_name = "description"
    pubdate_name = "published date"
    pubdate_format = "%a, %d %b %Y %H:%M:%S %Z"
    publisher_name = "publisher"
    news_outlet_name = "title"
    
    def __init__(self, query: str, start_date: datetime, end_date: datetime) -> None:
        self.query = query
        self.start_date = start_date
        self.end_date = end_date
        self.max_results = None
        self.query_metadata = [{}]

    @staticmethod
    def _format_metadata(metadata: list[dict]) -> list[dict]:
        formated_metadata = []
        
        for raw_single_metadata in metadata:
            single_metadata = {}
            single_metadata["title"] = raw_single_metadata[NewsFinder.title_name]
            single_metadata["description"] = raw_single_metadata[NewsFinder.description_name]
            single_metadata["pubdate"] = datetime.strptime(
                raw_single_metadata[NewsFinder.pubdate_name],
                NewsFinder.pubdate_format
            )
            publisher_info = raw_single_metadata[NewsFinder.publisher_name]
            single_metadata["publisher"] = publisher_info[NewsFinder.news_outlet_name]
            single_metadata["url"] = raw_single_metadata[NewsFinder.url_name]
            formated_metadata.append(single_metadata)
            
        return formated_metadata
            
    def get_metadata(self, max_results: int=Optional[None]) -> None:
        google_news = GNews(
            language=LANGUAGE,
            country=COUNTRY,
            start_date=start_date,
            end_date=end_date,
            max_results=max_results,
            #exclude_websites=['yahoo.com', 'cnn.com'],
        )
        query_metadata = google_news.get_news(self.query)
        self.query_metadata = NewsFinder._format_metadata(query_metadata)
        return self.query_metadata

    def save_pickle(self, path: Path) -> None:
        assert self.query_metadata != [{}], "run get_metadata first"

        now = datetime.now().start_date.strftime("%Y%m%d")
        start_date_str = self.start_date.strftime("%Y%m%d")
        end_date_str = self.end_date.strftime("%Y%m%d")
        pickle_fn = f"period_{start_date_str}-{end_date_str}"
        if self.max_results:
            pickle_fn += str(self.max_results)
        pickle_fn += f"-record_{now}"

        path_fn = path / self.query.lower().replace(" ", "_") / pickle_fn
        with open(path_fn, "wb") as file:
            pickle.dump(self.query_metadata, file)

    def load_pickle(self, path: Path):
        pickle_fn = f"{self.query.replace(" ", "_")}-{start_date_str}-{end_date_str}"
        with open(path_fn, "rb") as file:
            pickle.load(file)


class NewsExtractor():
    @staticmethod
    def _extract_content(article: Article, metadata: dict) -> dict:
        news = deepcopy(metadata)
        article.nlp()
        news["summary"] = article.summary
        news["keywords"] = article.keywords
        news["text"] = article.text
        return news
        
    def get_article(self, metadata: list[dict]) -> dict[str, str|datetime]:

        news = []
        for single_metadata in metadata:
            article = Article(url=single_metadata["url"], language=LANGUAGE)
            article.download()
            article.parse()
            if self.query in article.text:
                news.append(self._extract_content(article))

        return news  

NameError: name 'Pathlib' is not defined

In [154]:
query = "'mariana mortágua'" # "'Pedro Nuno Santos'"
news_finder = NewsFinder()
news_finder.get_metadata(query, (2023, 12, 1), (2023, 12, 5), 3)
display(news_finder.query_metadata)
news_finder.get_article()

[{'url': 'https://news.google.com/rss/articles/CBMie2h0dHBzOi8vd3d3LnRzZi5wdC9wcm9ncmFtYS9lbS1hbHRhLXZvei9lbWlzc2FvL21hcmlhbmEtbW9ydGFndWEtLS1jb29yZGVuYWRvcmEtbmFjaW9uYWwtZG8tYmxvY28tZGUtZXNxdWVyZGEtMTc0MzQyODguaHRtbNIBAA?oc=5&hl=en-US&gl=US&ceid=US:en',
  'title': 'Em Alta Voz - Mariana Mortágua - Coordenadora Nacional do Bloco ... - TSF Online',
  'description': 'Em Alta Voz - Mariana Mortágua - Coordenadora Nacional do Bloco ...  TSF Online',
  'pubdate': datetime.datetime(2023, 12, 2, 10, 17, 59),
  'publisher': 'TSF Online'},
 {'url': 'https://news.google.com/rss/articles/CBMib2h0dHBzOi8vd3d3LmRuLnB0L3BvbGl0aWNhL21hcmlhbmEtbW9ydGFndWEtc2VqYS1xdWVtLWZvci1vLWxpZGVyLWRvLXBzLXRlcmEtZGUtZW5mcmVudGFyLW8tYmxvY28tMTc0MzQzOTQuaHRtbNIBc2h0dHBzOi8vd3d3LmRuLnB0L3BvbGl0aWNhL2FtcC9tYXJpYW5hLW1vcnRhZ3VhLXNlamEtcXVlbS1mb3Itby1saWRlci1kby1wcy10ZXJhLWRlLWVuZnJlbnRhci1vLWJsb2NvLTE3NDM0Mzk0Lmh0bWw?oc=5&hl=en-US&gl=US&ceid=US:en',
  'title': 'Mariana Mortágua: "Seja quem for o líder do PS, terá de enf

In [75]:
article = google_news.get_full_article(news_json[0]['url']) 

In [176]:
datetime(2023, 12, 1).strftime("%Y-%m-%d")

'2023-12-01'

In [167]:
print(news_finder.query_metadata[1]["url"])
print(news_finder.query_metadata[1]["description"])

https://news.google.com/rss/articles/CBMib2h0dHBzOi8vd3d3LmRuLnB0L3BvbGl0aWNhL21hcmlhbmEtbW9ydGFndWEtc2VqYS1xdWVtLWZvci1vLWxpZGVyLWRvLXBzLXRlcmEtZGUtZW5mcmVudGFyLW8tYmxvY28tMTc0MzQzOTQuaHRtbNIBc2h0dHBzOi8vd3d3LmRuLnB0L3BvbGl0aWNhL2FtcC9tYXJpYW5hLW1vcnRhZ3VhLXNlamEtcXVlbS1mb3Itby1saWRlci1kby1wcy10ZXJhLWRlLWVuZnJlbnRhci1vLWJsb2NvLTE3NDM0Mzk0Lmh0bWw?oc=5&hl=en-US&gl=US&ceid=US:en
Mariana Mortágua: "Seja quem for o líder do PS, terá de enfrentar o Bloco"  Diário de Notícias


In [168]:
article = Article(url=news_finder.query_metadata[1]["url"], language=LANGUAGE)
article.download()
article.parse()
article.summary

''

In [170]:
print(article.text)

Tinha 27 anos quando foi eleita deputada pela primeira vez, já se disse perseguida por ser mulher, de esquerda, filha de um resistente antifascista e aparentemente por ter o dom de incomodar algumas pessoas com muito poder. É doutorada em Economia e Francisco Louçã já disse que daria uma boa ministra das Finanças. Lidera o Bloco de Esquerda desde maio passado.

Quem são os facilitadores com quem quer acertar contas que são, como diz, a nata podre do regime?

A nata podre do regime é um conjunto de interesses económicos que são as grandes empresas do país, que tiveram diferentes protagonistas ao longo do tempo. Alguns acabaram por ir à falência e arrastar com eles milhares de milhões em negócios e até em projetos importantes para o país. Outros surgiram agora na forma de grandes fundos de investimento, de grandes interesses e que têm depois quem lhes facilita a vida. E quem lhes facilita a vida é quem facilita o investimento, quem abre as portas, quem muda as leis a seu favor. Vimos iss

In [171]:
article.nlp()

In [173]:
print(article.summary)

Para um entendimento do Bloco com o PS é uma linha vermelha parar a privatização da TAP?
Eu sou líder do Bloco de Esquerda e candidato-me a eleições pelo meu programa.
Que não é o mesmo programa do PS e não é o mesmo programa da direita.
Sou líder do Bloco de Esquerda e vou debater com o próximo secretário-geral do PS.
Seja quem for o secretário-geral do PS, terá de enfrentar o Bloco de Esquerda sobre o que fez à habitação e ao Serviço Nacional de Saúde.


In [174]:
article.keywords

['extremadireita',
 'acho',
 'programa',
 'social',
 'mortágua',
 'esquerda',
 'bloco',
 'ps',
 'enfrentar',
 'país',
 'mariana',
 'força',
 'líder',
 'portugal']